In [1]:
from recipes_predict import RecipeInstance
from recipes_data_preprocessing import *

/data/anaconda3/envs/ghazaleh-dis/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-10-19 08:42:38.229939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-10-19 08:42:38.230065: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-19 08:42:38.230080: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install

In [6]:
def evaluate_instance(gold, predicted):
    fp, fn, tp, tn = 0, 0, 0, 0
    all_key_value_pairs = [('composition', 'change'), ('cookedness', 'cooked'), ('temperature', 'cold'), ('temperature', 'hot'), ('temperature', 'room'), ('temperature', 'nc'), ('rotation', 'turned'), ('rotation', 'nc'), ('shape', 'deformed'), ('shape', 'hit'), ('shape', 'molded'), ('shape', 'separated'), ('cleanliness', 'clean'), ('cleanliness', 'dry'), ('accessibility', 'not_accessible')]
    # true gold  >> true positive and false negative  
    for k, v in gold.items():
        for vi in v:
            if k in predicted.keys():
                if vi in predicted[k]:
                    tp += 1
                else:
                    fn += 1
            else:
                fn += 1
    for kv in all_key_value_pairs:
        k, v = kv
        if k not in gold.keys() and k not in predicted.keys():
            tn += 1            
    # also take care of location true negatives
    if 'location' not in gold.keys() and 'location' not in predicted.keys():
        tn += 1

    # false positives
    for k, v in predicted.items():
        if k not in gold.keys():
            fp += 1
        else:
            for vi in v:
                if vi not in gold[k]:
                    fp += 1    
    return {'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn}

def evaluate_location_only_instance(gold, predicted):
    fp, fn, tp, tn = 0, 0, 0, 0
    # all_key_value_pairs = [('composition', 'change'), ('cookedness', 'cooked'), ('temperature', 'cold'), ('temperature', 'hot'), ('temperature', 'room'), ('temperature', 'nc'), ('rotation', 'turned'), ('rotation', 'nc'), ('shape', 'deformed'), ('shape', 'hit'), ('shape', 'molded'), ('shape', 'separated'), ('cleanliness', 'clean'), ('cleanliness', 'dry'), ('accessibility', 'not_accessible')]
    # true gold  >> true positive and false negative  
    for k, v in gold.items():
        if k == 'location':
            for vi in v:            
                if k in predicted.keys():
                    if vi in predicted[k]:
                        tp += 1
                    else:
                        fn += 1
                else:
                    fn += 1        
    if 'location' not in gold.keys() and 'location' not in predicted.keys():
        tn += 1
    for k, v in predicted.items():
        if k == 'location':
            if k not in gold.keys():
                fp += 1
            else:
                for vi in v:
                    if vi not in gold[k]:
                        fp += 1    
    return {'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn}

def evaluate_split(split, split_data, location_only=False):
    fp, fn, tp, tn = 0, 0, 0, 0
    for i in range(len(split_data)):
        inst = RecipeInstance(split, split_data, i)
        for sid in inst.sentence_level.keys():
            gold = inst.sentence_level[sid]['sentence_gold_states']
            predicted = inst.sentence_level[sid]['sentence_predicted_states']
            if location_only:
                instance_eval_scores = evaluate_location_only_instance(gold, predicted)
            else:
                instance_eval_scores = evaluate_instance(gold, predicted)
            fp += instance_eval_scores['fp']
            fn += instance_eval_scores['fn']
            tp += instance_eval_scores['tp']
            tn += instance_eval_scores['tn']
    total = tp + tn + fp + fn
    precision = tp*1.0/float(tp+fp)
    recall = tp*1.0/float(tp+fn)
    f1 = 2*(recall*precision)/float(recall+precision)
    accuracy = (tp+tn)*1.0/float(total)
    return {'p': precision, 'r': recall, 'f1': f1, 'acc': accuracy}




In [7]:
split = 'test'
split_data = getSplit(split)
res = evaluate_split(split, split_data, location_only=True)
res

/bin/sh: 1: %20sugar%20,%20pour%20over%20pork%20.: not found
/bin/sh: 1: Syntax error: Unterminated quoted string


Parsing failed!


/bin/sh: 1: Syntax error: Unterminated quoted string


Parsing failed!
Parsing failed!


/bin/sh: 1: Syntax error: Unterminated quoted string
/bin/sh: 1: Syntax error: Unterminated quoted string


...

/bin/sh: 1: %20pat%20dry%20.: not found
/bin/sh: 1: Syntax error: Unterminated quoted string


Parsing failed!


/bin/sh: 1: %20chop%20tomatoes%20,%20discarding%20seeds%20.: not found
/bin/sh: 1: %20garlic%20and%20saute%20until%20onion%20is%20limp%20.: not found
/bin/sh: 1: %20pepper%20.: not found
/bin/sh: 1: %20: not found
/bin/sh: 1: Syntax error: EOF in backquote substitution


Parsing failed!


{'p': 0.3735408560311284,
 'r': 0.38247011952191234,
 'f1': 0.3779527559055118,
 'acc': 0.6400911161731208}